In [1]:
!echo $LD_LIBRARY_PATH

/usr/local/cuda-9.0/lib64/


In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.python.keras.backend import set_session
import numpy as np
import glob
import os
from tqdm import tqdm
%aimport utils

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
def vgg_layers(layer_names):
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

In [6]:
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

In [7]:
image_files = glob.glob("/mnt/workspace/Ugallery/images/*.jpg")

In [8]:
artwork_ids = [int(f[f.rfind('/')+1:-4]) for f in image_files]

In [9]:
N = len(artwork_ids)
N

13297

In [10]:
raw_images = np.empty((N, 224, 224, 3), dtype=np.float32)
for i in tqdm(range(N)):
    img_file = image_files[i]
    img = image.load_img(img_file, target_size=(224, 224), interpolation='bicubic')
    raw_images[i] = image.img_to_array(img)

100%|██████████| 13297/13297 [01:36<00:00, 137.80it/s]


In [11]:
raw_images.shape

(13297, 224, 224, 3)

In [14]:
batch_size = 64

In [15]:
gm_feat_dict = dict()

sess = tf.Session()
graph = tf.get_default_graph()

style_layers = ['block1_conv1',
                'block2_conv1']

with graph.as_default():
    with sess.as_default():
        
        style_extractor = vgg_layers(style_layers)
        
        conv_feat_tensor = tf.placeholder(shape=[None, None, None, None], dtype=tf.float32)
        gram_mat_tensor = gram_matrix(conv_feat_tensor)        
        
        for offset in tqdm(range(0, N, batch_size)):
            
            actual_batch_size = min(batch_size, N - offset)
            conv_features = style_extractor.predict(raw_images[offset : offset + actual_batch_size])
            
            gm_features = [
                sess.run(gram_mat_tensor, feed_dict={conv_feat_tensor: conv_feat})
                for conv_feat in conv_features
            ]
            
            assert len(gm_features) == len(style_layers)
            
            for sl, gmf in zip(style_layers, gm_features):
                try:
                    gm_feats = gm_feat_dict[sl]
                except KeyError:
                    gm_feats = gm_feat_dict[sl] = np.empty((N, *gmf.shape[1:]), dtype=np.float32)
                    print('gm_feat_dicts[%s] created with shape='%sl, gm_feats.shape)
                gm_feats[offset : offset + actual_batch_size] = gmf

  0%|          | 1/208 [00:01<06:00,  1.74s/it]

gm_feat_dicts[block1_conv1] created with shape= (13297, 64, 64)
gm_feat_dicts[block2_conv1] created with shape= (13297, 128, 128)


100%|██████████| 208/208 [05:40<00:00,  1.64s/it]


In [22]:
for key in gm_feat_dict.keys():
    gm_feat_dict[key] = gm_feat_dict[key].reshape(N, -1)

In [31]:
gm_feat_dict['block1_conv1'].std()

5540.252

In [32]:
dir_path = "/mnt/workspace/Ugallery/gram_matrix/"
os.makedirs(dir_path, exist_ok=True)
for key, gmf in gm_feat_dict.items():
    gmf.dump(dir_path + "%s.npy" % key)
with open(dir_path + 'ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)